<a href="https://colab.research.google.com/github/hykim-1/Study/blob/main/CIFAR_10_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# CIFAR-10: 3개 클래스(개, 고양이, 말)만 골라 CNN 분류
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix

# 1) 데이터 로드 (32x32x3, 라벨 0~9)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
y_train = y_train.ravel()
y_test = y_test.ravel()

# 2) 필요한 클래스만 필터링: 개(5), 고양이(3), 말(7)
keep = {5:0, 3:1, 7:2}  # 원본라벨 -> 새라벨 매핑 (dog=0, cat=1, horse=2)

train_mask = np.isin(y_train, list(keep.keys()))
test_mask  = np.isin(y_test,  list(keep.keys()))

x_train, y_train = x_train[train_mask], y_train[train_mask]
x_test,  y_test  = x_test[test_mask],  y_test[test_mask]

# 라벨 재매핑
y_train = np.vectorize(keep.get)(y_train)
y_test  = np.vectorize(keep.get)(y_test)

class_names = ['dog', 'cat', 'horse']

print("훈련셋:", x_train.shape, y_train.shape, {c:int(np.sum(y_train==c)) for c in range(3)})
print("테스트셋:", x_test.shape,  y_test.shape,  {c:int(np.sum(y_test==c))  for c in range(3)})

# 3) 전처리: 정규화 (0~1)
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

# 4) 간단한 CNN 모델
model = models.Sequential([
    layers.Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    layers.GlobalAveragePooling2D(),          # 파라미터 수를 줄이는 깔끔한 풀링
    layers.Dropout(0.3),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(3, activation='softmax')     # 3개 클래스 (dog, cat, horse)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 5) 학습
history = model.fit(
    x_train, y_train,
    epochs=15, batch_size=128,
    validation_split=0.1,
    verbose=2
)

# 6) 평가
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f"\n✅ Test Accuracy: {test_acc:.4f}")

# 7) 예측/리포트
pred_prob = model.predict(x_test, verbose=0)
y_pred = np.argmax(pred_prob, axis=1)

print("\n[Classification Report]")
print(classification_report(y_test, y_pred, target_names=class_names))

print("[Confusion Matrix] (rows=true, cols=pred)")
print(confusion_matrix(y_test, y_pred))


훈련셋: (15000, 32, 32, 3) (15000,) {0: 5000, 1: 5000, 2: 5000}
테스트셋: (3000, 32, 32, 3) (3000,) {0: 1000, 1: 1000, 2: 1000}
Epoch 1/15
106/106 - 33s - 311ms/step - accuracy: 0.3877 - loss: 1.0755 - val_accuracy: 0.4907 - val_loss: 0.9925
Epoch 2/15
106/106 - 38s - 360ms/step - accuracy: 0.5105 - loss: 0.9424 - val_accuracy: 0.5193 - val_loss: 0.9443
Epoch 3/15
106/106 - 41s - 385ms/step - accuracy: 0.5585 - loss: 0.8837 - val_accuracy: 0.5800 - val_loss: 0.8459
Epoch 4/15
106/106 - 41s - 383ms/step - accuracy: 0.5784 - loss: 0.8439 - val_accuracy: 0.6040 - val_loss: 0.8308
Epoch 5/15
106/106 - 26s - 244ms/step - accuracy: 0.6007 - loss: 0.8141 - val_accuracy: 0.6127 - val_loss: 0.8254
Epoch 6/15
106/106 - 26s - 241ms/step - accuracy: 0.6176 - loss: 0.7908 - val_accuracy: 0.6360 - val_loss: 0.7744
Epoch 7/15
106/106 - 42s - 398ms/step - accuracy: 0.6324 - loss: 0.7656 - val_accuracy: 0.6580 - val_loss: 0.7404
Epoch 8/15
106/106 - 26s - 244ms/step - accuracy: 0.6470 - loss: 0.7489 - val_acc